# Tiesinė regresija užduotis

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
fish = pd.read_csv('https://raw.githubusercontent.com/robotautas/kursas/master/Machine%20Learning/Fish.csv')

In [4]:
fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


### Užduotis
Turime lentelę su 7 žuvų rūšim, bei įvairiais jų išmatavimais. Vienas iš jų yra svoris - tai bus mūsų *target* stulpelis. **Sukurkite tiesinės regresijos modelį, kuris pagal kitus lentelės stulpelius spėlios žuvies svorį.** Išsiaiškinkite, ar yra trūkstamų reikšmių, atsispausdinkite keletą diagramų ir histogramų, patyrinėkite tendencijas ir koreliacijas. Kilus neaiškumams, remkitės paskaitos medžiaga. Sėkmės!


In [5]:
fish.info

<bound method DataFrame.info of     Species  Weight  Length1  Length2  Length3   Height   Width
0     Bream   242.0     23.2     25.4     30.0  11.5200  4.0200
1     Bream   290.0     24.0     26.3     31.2  12.4800  4.3056
2     Bream   340.0     23.9     26.5     31.1  12.3778  4.6961
3     Bream   363.0     26.3     29.0     33.5  12.7300  4.4555
4     Bream   430.0     26.5     29.0     34.0  12.4440  5.1340
..      ...     ...      ...      ...      ...      ...     ...
154   Smelt    12.2     11.5     12.2     13.4   2.0904  1.3936
155   Smelt    13.4     11.7     12.4     13.5   2.4300  1.2690
156   Smelt    12.2     12.1     13.0     13.8   2.2770  1.2558
157   Smelt    19.7     13.2     14.3     15.2   2.8728  2.0672
158   Smelt    19.9     13.8     15.0     16.2   2.9322  1.8792

[159 rows x 7 columns]>

In [6]:
dummies = pd.get_dummies(fish['Species']) # padarem, kad butu 1 / 0 'Species'
data = pd.concat([fish, dummies], axis=1)
data.drop('Species', axis=1, inplace=True)

In [7]:
data.columns
x = data[['Length1', 'Length2', 'Length3', 'Height', 'Width', 'Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']]
y = data['Weight']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
reg = LinearRegression().fit(x_train, y_train)


In [11]:
reg.score(x_test, y_test)

0.9405192628555377

In [12]:
preds = reg.predict(x_test)

In [13]:
new_column = pd.Series(data=preds, name='spejimas')
new_column.head(15)


0       25.241369
1        0.595421
2      208.449859
3      331.665261
4      215.482994
5      753.929942
6      -72.458507
7      253.698723
8      259.678480
9     1186.485729
10     600.206027
11     847.635320
12     537.239650
13     152.193053
14     678.093881
Name: spejimas, dtype: float64

In [14]:
res = pd.concat([y_test.reset_index(), new_column], axis=1)[['Weight', 'spejimas']]
res.head()

,Weight,spejimas
0,78.0,25.241369
1,13.4,0.595421
2,200.0,208.449859
3,270.0,331.665261
4,150.0,215.482994


In [15]:
res['klaida']= res['Weight'] - res['spejimas']
res.head()

,Weight,spejimas,klaida
0,78.0,25.241369,52.758631
1,13.4,0.595421,12.804579
2,200.0,208.449859,-8.449859
3,270.0,331.665261,-61.665261
4,150.0,215.482994,-65.482994


In [16]:
sns.reset_defaults()
sns.histplot(res['klaida'])


<AxesSubplot:xlabel='klaida', ylabel='Count'>

In [17]:
su_pavadinimais = pd.concat([fish, res])
su_pavadinimais

,Species,Weight,Length1,Length2,Length3,Height,Width,spejimas,klaida
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200,NaN,NaN
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056,NaN,NaN
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961,NaN,NaN
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555,NaN,NaN
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340,NaN,NaN
...,...,...,...,...,...,...,...,...,...
48,NaN,130.0,NaN,NaN,NaN,NaN,NaN,135.643024,-5.643024
49,NaN,85.0,NaN,NaN,NaN,NaN,NaN,70.255777,14.744223
50,NaN,685.0,NaN,NaN,NaN,NaN,NaN,720.492577,-35.492577
51,NaN,500.0,NaN,NaN,NaN,NaN,NaN,485.662042,14.337958
